In [1]:
import os
import pandas as pd
import boto3

In [2]:
# Get AWS credentials from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
# Set up S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [5]:
# Define the S3 bucket and CSV files
bucket_name = 'mindex-data-analytics-code-challenge'
bengals = 'bengals.csv'
boyd_receiving = 'boyd_receiving.csv'
chase_receiving = 'chase_receiving.csv'
higgins_receiving = 'higgins_receiving.csv'

In [6]:
# Download the CSV files
s3.download_file(bucket_name, bengals, bengals)
s3.download_file(bucket_name, boyd_receiving, boyd_receiving)
s3.download_file(bucket_name, chase_receiving, chase_receiving)
s3.download_file(bucket_name, higgins_receiving, higgins_receiving)

In [32]:
bengals_df = pd.read_csv(bengals)
boyd_receiving_df = pd.read_csv(boyd_receiving)
chase_receiving_df = pd.read_csv(chase_receiving)
higgins_receiving_df = pd.read_csv(higgins_receiving)

In [44]:
boyd_receiving_df.head()

,Week,Yards,TD
0,REG1,32,0
1,REG2,73,0
2,REG3,36,1
3,REG4,118,0
4,REG5,24,0


In [66]:
# Join and rename shared columns with player names in suffixes
result_df = pd.merge(bengals_df, boyd_receiving_df, on='Week', how='inner')
result_df = pd.merge(result_df, chase_receiving_df, on='Week', how='inner', suffixes=('_boyd', '_chase'))
result_df = pd.merge(result_df, higgins_receiving_df, on='Week', how='inner')
result_df = result_df.rename(columns={'Yards': 'Yards_higgins', 'TD': 'TD_higgins'})

# replace game 'Result' values 0.0 and 1.0 with 'Loss' and 'Win', respectively
result_df['Result'] = result_df['Result'].replace({0.0: 'Loss', 1.0: 'Win'})

In [67]:
result_df.head()

,Week,Opponent,Location,Result,Yards_boyd,TD_boyd,Yards_chase,TD_chase,Yards_higgins,TD_higgins
0,REG1,MIN,Home,Win,32,0,101,1,58,1
1,REG2,CHI,Away,Loss,73,0,54,1,60,1
2,REG5,GB,Home,Loss,24,0,159,1,32,0
3,REG6,DET,Away,Win,7,0,97,0,44,0
4,REG7,BAL,Away,Win,39,0,201,1,62,0
